This is a Jupyter notebook that is used to play around with code for sfeature extraction into a pandas dataframe. Then, I will manipulate the dataframe to extract the gpus nodes to run for a specific type of gpu.

In [26]:
import pandas as pd
import os
import re
import subprocess
from io import StringIO

In [27]:
# Create text file of the sfeature output
# os.system("sfeature | awk '/CCV Notes/ {exit} {print}' > sfeature.txt")
# os.system(r'sinfo -o "%N %P %c %m %f %G" > sfeature.txt')
# result = subprocess.run(["(exit 1)"], shell=True, capture_output=True, text=True, check=True)
try:
    result = subprocess.run(args=["sinfo", "-o", r"%N %P %c %m %f %G"], shell=False, capture_output=True, text=True, check=True)
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print("Error", e)
    raise e

NODELIST PARTITION CPUS MEMORY AVAIL_FEATURES GRES
node[2405-2413,2415,2421-2424] batch* 192 1540000 192core,amd,genoa,edr (null)
node[2417-2420] batch* 64 514000 64core,intel,icelake,edr (null)
node[1317-1328] batch* 24 382000+ 24core,intel,e5-2670,e5-2600,scalable,skylake,fdr (null)
node[1613-1656,1701-1756] batch* 32 190000 32core,intel,scalable,cascade,edr (null)
node[1804-1820,1833-1856,1904-1956,2301-2352] batch* 48 382000 48core,intel,cascade,edr (null)
node[1333-1364,1601-1608] batch* 32 382000 32core,intel,scalable,cascade,edr,cifs (null)
node[2405-2413,2415,2421-2424] dbgap 192 1540000 192core,amd,genoa,edr (null)
node[2417-2420] dbgap 64 514000 64core,intel,icelake,edr (null)
node[1317-1328] dbgap 24 382000+ 24core,intel,e5-2670,e5-2600,scalable,skylake,fdr (null)
node[1613-1656,1701-1756] dbgap 32 190000 32core,intel,scalable,cascade,edr (null)
node[1804-1820,1833-1856,1904-1956,2301-2352] dbgap 48 382000 48core,intel,cascade,edr (null)
node[1333-1364,1601-1608] dbgap 32 38

In [28]:
# Read the file into a pandas dataframe
# df = pd.read_table("sfeature.txt", delimiter=' ')
df = pd.read_table(StringIO(result.stdout), delimiter=' ')
print(df.columns)
df.head()

Index(['NODELIST', 'PARTITION', 'CPUS', 'MEMORY', 'AVAIL_FEATURES', 'GRES'], dtype='object')


,NODELIST,PARTITION,CPUS,MEMORY,AVAIL_FEATURES,GRES
0,"node[2405-2413,2415,2421-2424]",batch*,192,1540000,"192core,amd,genoa,edr",(null)
1,node[2417-2420],batch*,64,514000,"64core,intel,icelake,edr",(null)
2,node[1317-1328],batch*,24,382000+,"24core,intel,e5-2670,e5-2600,scalable,skylake,fdr",(null)
3,"node[1613-1656,1701-1756]",batch*,32,190000,"32core,intel,scalable,cascade,edr",(null)
4,"node[1804-1820,1833-1856,1904-1956,2301-2352]",batch*,48,382000,"48core,intel,cascade,edr",(null)


In [10]:
df.loc[df["PARTITION"] == "gracehopper"]

,NODELIST,PARTITION,CPUS,MEMORY,AVAIL_FEATURES,GRES
53,gpu2706,gracehopper,72,530000,"grace,hopper",gpu:gh200:1(S:0)
54,gpu2707,gracehopper,72,530000,"grace,hopper,mig","gpu:nvidia_gh200_480gb_2g.24gb:3(S:0),gpu:nvid..."


In [11]:
# Extract the gpus
gpus_df = df.loc[df["PARTITION"] == "gpu"]
gpus_df.reset_index(drop=True, inplace=True)
gpus_df

,NODELIST,PARTITION,CPUS,MEMORY,AVAIL_FEATURES,GRES
0,gpu2262,gpu,64,1028000,"amd,gpu,a5000,ampere",gpu:a5000:2(S:0)
1,gpu2112,gpu,64,1028000,"amd,gpu,geforce3090,ampere",gpu:geforce3090:7(S:0-1)
2,"gpu[1414,2001-2010]",gpu,32+,382000,"intel,gpu,quadrortx,turing,cascade",gpu:quadrortx:10(S:0)
3,gpu[1401-1403],gpu,24+,190000+,"intel,gpu,titanrtx,turing,skylake,6142",gpu:titanrtx:5(S:0)
4,"gpu[2101-2111,2113-2116,2264]",gpu,48+,1028000,"amd,gpu,geforce3090,ampere",gpu:geforce3090:8(S:0-1)
5,"gpu[2250-2251,2501-2505]",gpu,128,1028000,"amd,cifs,gpu,a5500,ampere",gpu:a5500:8(S:0-1)
6,"gpu[2252-2260,2263,2601-2610]",gpu,64+,1028000,"amd,cifs,gpu,a5000,ampere",gpu:a5000:8(S:0-1)
7,gpu[2506-2507],gpu,128,1028000,"amd,gpu,a5500,cifs,ampere",gpu:a5500:8(S:0-1)
8,gpu[2508-2510],gpu,128,1028000,"amd,gpu,cifs,a5000,ampere",gpu:a5000:8(S:0-1)


In [12]:
# Get the nodelist column
nodelists = gpus_df["NODELIST"]
nodelists

0                          gpu2262
1                          gpu2112
2              gpu[1414,2001-2010]
3                   gpu[1401-1403]
4    gpu[2101-2111,2113-2116,2264]
5         gpu[2250-2251,2501-2505]
6    gpu[2252-2260,2263,2601-2610]
7                   gpu[2506-2507]
8                   gpu[2508-2510]
Name: NODELIST, dtype: object

In [15]:
# Get a list of all the nodes
pattern = re.compile(r'\[([^\[\]]+)\]') # Regex for strings enclosed in square brackets
node_node_list = []
for nodelist in nodelists:
    node_list = []
    stripped_node = nodelist.strip("gpu")
    match = pattern.search(stripped_node)
    if match:
        enclosed_string = match.group(1)
        items = enclosed_string.split(',')

        for item in items:
            if '-' in item:
                start, end = map(int, item.split('-'))
                node_list.extend(range(start, end + 1))
            else:
                node_list.append(int(item))
    else:
        node_list.append(int(stripped_node))
    node_node_list.extend(node_list)
    print(node_list)

[2262]
[2112]
[1414, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]
[1401, 1402, 1403]
[2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2113, 2114, 2115, 2116, 2264]
[2250, 2251, 2501, 2502, 2503, 2504, 2505]
[2252, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2263, 2601, 2602, 2603, 2604, 2605, 2606, 2607, 2608, 2609, 2610]
[2506, 2507]
[2508, 2509, 2510]


In [16]:
# Print all the gpu nodes in one list
print(node_node_list)

[2262, 2112, 1414, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 1401, 1402, 1403, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2113, 2114, 2115, 2116, 2264, 2250, 2251, 2501, 2502, 2503, 2504, 2505, 2252, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2263, 2601, 2602, 2603, 2604, 2605, 2606, 2607, 2608, 2609, 2610, 2506, 2507, 2508, 2509, 2510]
